In [1]:
from os import listdir
from os.path import isfile, join
import json
import math
import ast
import openpyxl
import numpy as np
import networkx as nx

In [2]:
import pandas as pd
import pickle

from mdutils.mdutils import MdUtils
from mdutils import Html



## Parameters

In [3]:
base_dir = join("Vaults", "aaaaSpringerNature")

#base_dir = "data"

dirr = join(base_dir, "05_model_input/")

md_dir = join(base_dir, "Texts_Vault")


num_results = 2

concept_count_for_author = 10

dbpedia_types = {'Person': True}

#"http://dbpedia.org/ontology/Person"


conceptsdir = join(base_dir, "04_feature")



## Read Files

In [4]:
raw_text_df = pd.read_csv(dirr+"text_df_final.csv", index_col=0)

text_df = pd.read_csv(dirr+"text_df_final.csv", index_col=0)

#text_df = pd.read_csv("data/01_raw/text_df.csv", index_col=0)

mentions_df = pd.read_csv(dirr+"mentions_df_final.csv")


mentions_df.sort_values("Start_pos", ascending=False, inplace=True)


relatedness_semantic_df = pd.read_csv(dirr+"relatedness_df.csv")
relatedness_semantic_df.index = relatedness_semantic_df['Text_ID']

relatedness_concepts_df = pd.read_csv(dirr+"relatedness_concepts_df.csv", index_col=0)

relatedness_concepts_df_repeat = pd.read_csv(dirr+"relatedness_concepts_df_repeat.csv", index_col=0)

#concepts_df = pd.read_csv(dirr+"concepts_database_enhanced.csv", index_col=0)

#concepts_df_conceptnet = pd.read_csv(dirr+"concepts_database_enhanced_conceptnet.csv", index_col=0)

with open(dirr+"conceptnet_to_dbpedia.json", "r") as f:
        conceptnet_to_dbpedia =  json.load(f)


with open(dirr+"concepts_list_final.json", "r") as f:
        concepts_list = json.load(f)


def add_title_column(text_df):
	title_column = []
	count_dict = {}
	for name in text_df['Father_name'].unique():
		count_dict[name.split(":")[0]] = 1

	for row in text_df.itertuples():
		name = row[2].split(":")[0]
		title = name + " Note " + str(count_dict[name])
		title_column.append(title)
		count_dict[name] += 1
	text_df['Title'] = title_column
	return text_df

text_df = add_title_column(text_df)

#text_df = add_title_column_book(text_df)

In [5]:
len(concepts_list)

152

In [6]:
conceptsdir = join(base_dir, "04_feature")

with open(join(conceptsdir,"concepts_data_list.json"), "r") as f:
        concepts_data_list = json.load(f)

with open(join(conceptsdir,"conceptnet_distances_dict.json"), "r") as f:
        conceptnet_distances_dict = json.load(f)

with open(join(conceptsdir,"conceptnet_relations_dict.json"), "r") as f:
        conceptnet_relations_dict = json.load(f)



new_concepts_data_list = []
for data in concepts_data_list:
	if type(data) != type([0]):
		print("Error")
		print(data)
	else:
		new_concepts_data_list.append(data)


column_names = ["DBpedia URL", "Name", "Abstract", "Image", "DBpedia Types", "Subject FROM", "Related Entities FROM", "Related Entities OF", "Subject OF", "Narrower Categories", "Broader Categories"] 

concept_df = pd.DataFrame(new_concepts_data_list, columns=column_names)

concepts_database = concept_df[concept_df["DBpedia URL"].isin(concepts_list)]


relations_dict_column = []
distances_dict_column = []

for row in concepts_database.itertuples():

	concept_id = row[1]

	distances_dict = conceptnet_distances_dict[concept_id]

	relations_dict = conceptnet_relations_dict[concept_id]

	distances_dict_column.append(distances_dict)

	relations_dict_column.append(relations_dict)

concepts_database_enhanced = concepts_database.copy()

concepts_database_enhanced['ConceptNet_Distances'] = distances_dict_column

concepts_database_enhanced['ConceptNet_Relations'] = relations_dict_column

concepts_df = concepts_database_enhanced

concepts_list = list(concepts_df["DBpedia URL"])
print(len(concepts_list))

concepts_df = concepts_df.set_index("DBpedia URL")


127


## Author Data

In [7]:
author_texts_dict = {}

author_list = text_df['Author_name'].unique()

for author in author_list:
	#Related Concepts:
	author_texts_dict[author] = []

for index, row in text_df.iterrows():
	text_author = row['Author_name']
	text_id = row["Text_ID"]
	author_texts_dict[text_author].append(str(text_id))


	#Related Authors


In [8]:



author_relatedness = {}
i = 0

max_relatedness_list = []
avg_relatedness_list = []

for author1 in author_list:
	j = 0
	author1_texts = author_texts_dict[author1]
	num_texts = len(author1_texts)
	row_max = []
	row_avg = []
	for author2 in author_list:
		if author1 == author2:
			final_average_relatedness = 1
			max_relatedness = 1
		else:
			author2_texts = author_texts_dict[author2]
			total_relatedness = 0
			max_relatedness = 0

			for text_id in author1_texts:
				average_relatedness = relatedness_semantic_df.loc[int(text_id)][author2_texts].mean()
				if average_relatedness > max_relatedness:
					max_relatedness = average_relatedness
				total_relatedness+=average_relatedness
			final_average_relatedness = total_relatedness/num_texts
		row_max.append(max_relatedness)
		
		row_avg.append(final_average_relatedness)

	max_relatedness_list.append(row_max)
	avg_relatedness_list.append(row_avg)


avg_relatedness_df = pd.DataFrame(avg_relatedness_list, index=author_list, columns=author_list)




In [9]:

author_concept_count = {}

for author in author_list:

	author_concept_count[author] = {}

for index, row in mentions_df.iterrows():
	mention_author = row['Author_name']
	concept = row["Entity Name"]

	if concept not in author_concept_count[mention_author].keys():

		author_concept_count[mention_author][concept] = 1
	else:
		author_concept_count[mention_author][concept] += 1


author_concept_dict = {}
for author in author_list:
	author_concepts = sorted(author_concept_count[author].items(), key=lambda item: item[1], reverse=True)
	author_concept_dict[author] = author_concepts

## Create 4 Graphs

In [10]:
import networkx as nx

G = nx.Graph()
G.add_nodes_from(list(text_df['Title']))
G.add_nodes_from(concepts_list)

GD = nx.DiGraph()
GD.add_nodes_from(list(text_df['Title']))
GD.add_nodes_from(concepts_list)



G_only_texts = nx.Graph()
G_only_texts.add_nodes_from(list(text_df['Title']))

GD_only_texts = nx.DiGraph()
GD_only_texts.add_nodes_from(list(text_df['Title']))



G_only_all_concepts = nx.Graph()
G_only_all_concepts.add_nodes_from(list(text_df['Title']))
G_only_all_concepts.add_nodes_from(concepts_list)
GD_only_all_concepts = nx.DiGraph()
GD_only_all_concepts.add_nodes_from(list(text_df['Title']))
GD_only_all_concepts.add_nodes_from(concepts_list)


G_only_text_concepts = nx.Graph()
G_only_text_concepts.add_nodes_from(list(text_df['Title']))
G_only_text_concepts.add_nodes_from(concepts_list)
GD_only_text_concepts = nx.DiGraph()
GD_only_text_concepts.add_nodes_from(list(text_df['Title']))
GD_only_text_concepts.add_nodes_from(concepts_list)

In [11]:
type_dict = {}

for text in list(text_df['Title']): type_dict[text] = {"type": "Text"}

for concept in concepts_list: type_dict[concept] = {"type": "Concept"}

#nx.set_node_attributes(G, type_dict)

## Manipulations Text Nodes

### Mentions Substitutions

In [12]:
#concepts_df[concepts_df['Name']=="Courage"]

In [13]:
def substitute_no_alias(text, entity, start, end):
	return text[0:start] + " [[" + entity + "]] " + text[end:]

def substitute_with_alias(text, entity, mention, start, end):
	return text[0:start] + " [[" + entity + "|" + mention + "]] " + text[end:]


def apply_substituitons(highlights_df, mentions_df):


	for row in mentions_df.itertuples():

		highlight_id = row[1]
		mention = row[6]
		start = row[7]
		end = row[8]
		entity = row[9]
		
		text = highlights_df.loc[highlight_id, "Text"]

		if "[[" + entity + "]]" in text[start:end+35] or mention + "]]" in text[start:end+35]:
			pass
		else:
			if mention.title() == entity.title():
				text = substitute_no_alias(text, entity, start, end)
			else:
				text = substitute_with_alias(text, entity, mention, start, end)
		# if highlight_id == 319799737:
		# 	print(highlights_df.loc[highlight_id, "Text"])
		# 	print("---\n", text)
		highlights_df.loc[highlight_id, "Text"] = text

	return highlights_df

text_df = apply_substituitons(text_df, mentions_df)


### Add Text Relationships

In [14]:
relatedness_semantic_df.head(3)

def create_related_texts_dataframe(semantic_similarities, num_results):
	
	recommendations_df_list = []

	for index, row in semantic_similarities.iterrows():

		idd = row["Text_ID"]
		#print(idd)
		query_author = row["Author_name"]

		column_name = str(idd)

		query_column_names = ["Text_ID", "Author_name", column_name]

		query_df = semantic_similarities[query_column_names].copy().sort_values(column_name, ascending=False).reset_index(drop=True)

		same_author_key_list = []
		different_author_key_list = []

		first = True
		for row2 in query_df.itertuples():
			key = row2[1]
			author_name = row2[2]
			score = row2[3]
			
			#distance = 0

			# Append a distance em algum lugar ↓

			if author_name == query_author and first == False:
				if len(same_author_key_list) < num_results:
					same_author_key_list.append({key:score})
			elif first == False:
				if len(different_author_key_list) < num_results:
					different_author_key_list.append({key:score})
			first = False
			
		recommendations_df_list.append([idd, same_author_key_list, different_author_key_list])
		
	semantic_recommendations_df = pd.DataFrame(recommendations_df_list, columns=["Text_ID", "Same Author Recs", "Diff Author Recs"])
	semantic_recommendations_df.set_index("Text_ID", inplace=True)

	return semantic_recommendations_df

In [15]:
semantic_recommendations = create_related_texts_dataframe(relatedness_semantic_df, num_results=num_results)

semantic_recommendations

,Same Author Recs,Diff Author Recs
Text_ID,,
110306476,"[{290432993: 0.91128}, {290432995: 0.5151}]","[{406891963: 0.51646}, {336303820: 0.50196}]"
110306477,"[{290432997: 0.49518}, {290432999: 0.47503}]","[{590584777: 0.48987}, {590635942: 0.45945}]"
110306479,"[{290432993: 0.55997}, {290432999: 0.49657}]","[{583516121: 0.57397}, {336303818: 0.46484}]"
110306480,"[{290432996: 0.51655}, {290433000: 0.446}]","[{336303816: 0.55601}, {590605099: 0.55483}]"
110306481,"[{376444506: 0.50613}, {376444505: 0.47604}]","[{463559130: 0.59345}, {583516114: 0.58909}]"
...,...,...
591548161,"[{591548159: 0.73799}, {591548154: 0.70139}]","[{376433767: 0.53204}, {583516114: 0.53143}]"
591548162,"[{547332530: 0.63187}, {547332529: 0.63018}]","[{583516124: 0.53747}, {583516121: 0.5209}]"
591548163,"[{591548165: 0.57666}, {514356011: 0.52415}]","[{583516121: 0.56975}, {376444506: 0.55311}]"


In [16]:
#concept_based_recommendations_repeat = create_related_texts_dataframe(relatedness_concepts_df_repeat, num_results=num_results)

concept_based_recommendations = create_related_texts_dataframe(relatedness_concepts_df, num_results=num_results)

concept_based_recommendations

,Same Author Recs,Diff Author Recs
Text_ID,,
110306476,"[{290432993: 0.4918069564908715}, {110306479: ...","[{590605099: 0.3904362057500617}, {590605101: ..."
110306479,"[{290432993: 0.4140740508369426}, {110306476: ...","[{591548151: 0.30280276816609}, {583516118: 0...."
110306480,"[{290433001: 0.6229012414727946}, {376444509: ...","[{336303818: 0.7828186265910537}, {496905538: ..."
290432993,"[{110306476: 0.4918069564908715}, {110306479: ...","[{590605099: 0.3759842150471769}, {590605101: ..."
290432995,"[{110306480: 0.5036375726620896}, {376444509: ...","[{336303818: 0.5471934385250757}, {496905538: ..."
...,...,...
591548161,"[{591548159: 0.4989619955033399}, {514356017: ...","[{376444501: 0.4977929051503558}, {376433767: ..."
591548162,"[{591548163: 0.5361312377537832}, {514356017: ...","[{406891963: 0.6024259551481475}, {336303815: ..."
591548163,"[{547332524: 0.6101140221580419}, {591548165: ...","[{376444509: 0.7996482294246017}, {590635944: ..."


# Build Knowledge Repository → Obsidian

## Criar Text Nodes

In [17]:
def get_title_for_highlight_id(id_dict):
	id = list(id_dict.keys())[0]
	try:
		title = text_df.loc[id, "Title"]
	except:
		title = ""
	
	return title, round(id_dict[id], 3)

	

## Execute Text Nodes

In [18]:
#mentions_df.head(2)
#text_df.head(3)

count_dict = {}
author_list = []

for row in text_df.itertuples():

	highlight_id = row[0]
	book = row[2].split(":")[0]
	author = row[3]
	text = row[4]
	#notes = row[5]
	title = row[6]
	print(title)
	
	if book not in list(count_dict.keys()):
		count_dict[book] = 1

		mdBook = MdUtils(file_name=join(md_dir, "Books",
		 book))

		mdBook.new_paragraph("---")
		mdBook.new_paragraph("tags: #Book")
		mdBook.new_paragraph("author: " + author.replace(" and ", " "))
		mdBook.new_paragraph("---")

		mdBook.new_header(level=1, title=book)

		mdBook.new_paragraph("- author " + "[[" + author + "]]")
		mdBook.create_md_file()

	if author not in list(author_list):
		author_list.append(author)
		num_texts_from_author = len(author_texts_dict[author])
		num_concepts = len(author_concept_dict[author])

		mdAuthor = MdUtils(file_name=join(md_dir, "Authors", author))

		mdAuthor.new_paragraph("---")
		mdAuthor.new_paragraph("tags: #Author")
		mdAuthor.new_paragraph("---")

		mdAuthor.new_header(level=1, title=author)
		mdAuthor.new_header(level=2, title="Author Overview")
		mdAuthor.new_paragraph("Total Texts: " + str(num_texts_from_author))
		mdAuthor.new_paragraph("Unique Concepts Used: " + str(num_concepts))
		mdAuthor.new_paragraph("Ratio of Concepts by Text: " + str(round(num_concepts/num_texts_from_author, 2)))

		mdAuthor.new_header(level=2, title="Most Used Concepts")
		for concept_tuple in author_concept_dict[author][0:concept_count_for_author]:
			concept = concept_tuple[0]
			counts = concept_tuple[1]
			mdAuthor.new_paragraph("- [[" +concept +"]] used " + str(counts) + " times, or " + str(round(counts/num_texts_from_author,2)) + " per text.")

		mdAuthor.new_header(level=2, title="Related Authors")
		related_authors = list(avg_relatedness_df.sort_values(by=author, ascending=False).index)

		for author_name in related_authors[1:num_results+1]:
			
			mdAuthor.new_paragraph("- relatedToAuthor [[" + author_name + "]]")

		mdAuthor.new_header(level=2, title="Texts from Author")

		for text_id in author_texts_dict[author]:
			title_text = text_df.loc[int(text_id)]["Title"]
			mdAuthor.new_paragraph("- hasText [[" + title_text+"]]")


		mdAuthor.create_md_file()
	
	mdHigh = MdUtils(file_name=join(md_dir, "Texts", title)) 

	mdHigh.new_paragraph("---")
	mdHigh.new_paragraph("tags: #Highlight")
	mdHigh.new_paragraph("author: [" + author.replace(" and ", " ") + "]")

	mdHigh.new_paragraph("---")
		
	mdHigh.new_header(level=1, title=title)
	mdHigh.new_paragraph("", wrap_width=0)
	mdHigh.new_paragraph(text, wrap_width=0)
	mdHigh.new_paragraph("", wrap_width=0)

	mdHigh.new_header(level=2, title="Recommended Highlights")
	
	
	try:
		mdHigh.new_header(level=3, title="Semantic Relatedness")
		#mdHigh.new_header(level=4, title="From the Same Author")
	
		rank = 1
		for hid in semantic_recommendations.loc[highlight_id, "Same Author Recs"]:
			#print(hid)
			title2, score = get_title_for_highlight_id(hid)
			text = "- sameauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
			rank +=1
			mdHigh.new_paragraph(text, wrap_width=0)

			G.add_edge(title, title2)
			G_only_texts.add_edge(title, title2)

			GD.add_edge(title, title2)
			GD_only_texts.add_edge(title, title2)

		#mdHigh.new_header(level=4, title="From Different Author")
		
		rank = 1
		for hid in semantic_recommendations.loc[highlight_id, "Diff Author Recs"]:
			title2, score = get_title_for_highlight_id(hid)
			text = "- diffauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
			rank +=1
			mdHigh.new_paragraph(text, wrap_width=0)

			G.add_edge(title, title2)
			G_only_texts.add_edge(title, title2)

			GD.add_edge(title, title2)
			GD_only_texts.add_edge(title, title2)
		
		
		mdHigh.new_header(level=3, title="Shared Concept Relatedness")

		#mdHigh.new_header(level=4, title="From the Same Author")
		rank = 1
		for hid in concept_based_recommendations.loc[highlight_id, "Same Author Recs"]:
			#print(hid)
			title2, score = get_title_for_highlight_id(hid)
			text = "- sameauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
			rank +=1
			mdHigh.new_paragraph(text, wrap_width=0)

			G.add_edge(title, title2)
			G_only_texts.add_edge(title, title2)

			GD.add_edge(title, title2)
			GD_only_texts.add_edge(title, title2)
			


		#mdHigh.new_header(level=4, title="From Different Author")
		rank = 1
		for hid in concept_based_recommendations.loc[highlight_id, "Diff Author Recs"]:
			#print(hid)
			title2, score = get_title_for_highlight_id(hid)
			text = "- diffauthor" + str(rank) + " [[" + title2 + "]] - " + str(score)
			rank +=1
			mdHigh.new_paragraph(text, wrap_width=0)

			G.add_edge(title, title2)
			G_only_texts.add_edge(title, title2)

			GD.add_edge(title, title2)
			GD_only_texts.add_edge(title, title2)

	except:
		mdHigh.new_paragraph("- No Concepts in the Text", wrap_width=0)
		pass
	
	

	mdHigh.new_header(level=1, title="Metadata")
	mdHigh.new_paragraph("- frombook [[" + book + "]]")
	mdHigh.new_paragraph("- fromauthor [[" + author + "]]")
	
	active_concepts = []
	
	for concept_row in mentions_df[mentions_df["Text_ID"]==highlight_id].itertuples():
		concept_name = concept_row[9]
		concept_url = concept_row[10]
		if concept_name not in active_concepts:
			
			mdHigh.new_paragraph("- hasconcept [[" + concept_name + "]]")
			active_concepts.append(concept_name)

			G.add_edge(title, concept_url)
			G_only_all_concepts.add_edge(title, concept_url)
			G_only_text_concepts.add_edge(title, concept_url)

			GD.add_edge(title, concept_url)
			GD_only_all_concepts.add_edge(title, concept_url)
			GD_only_text_concepts.add_edge(title, concept_url)

			GD.add_edge(concept_url, title)
			GD_only_all_concepts.add_edge(concept_url, title)
			GD_only_text_concepts.add_edge(concept_url, title)
	
	
	mdHigh.create_md_file()



How to Take Smart Notes Note 1
How to Take Smart Notes Note 2
How to Take Smart Notes Note 3
How to Take Smart Notes Note 4
How to Take Smart Notes Note 5
How to Take Smart Notes Note 6
How to Take Smart Notes Note 7
How to Take Smart Notes Note 8
How to Take Smart Notes Note 9
How to Take Smart Notes Note 10
How to Take Smart Notes Note 11
How to Take Smart Notes Note 12
How to Take Smart Notes Note 13
How to Take Smart Notes Note 14
Building a Second Brain Note 1
Building a Second Brain Note 2
Building a Second Brain Note 3
Building a Second Brain Note 4
Building a Second Brain Note 5
Building a Second Brain Note 6
Building a Second Brain Note 7
Building a Second Brain Note 8
Building a Second Brain Note 9
Building a Second Brain Note 10
Building a Second Brain Note 11
Building a Second Brain Note 12
Building a Second Brain Note 13
Building a Second Brain Note 14
Building a Second Brain Note 15
Building a Second Brain Note 16
Building a Second Brain Note 17
Building a Second Brain No

## Add Concepts Links

In [19]:
concepts_df.head()

,Name,Abstract,Image,DBpedia Types,Subject FROM,Related Entities FROM,Related Entities OF,Subject OF,Narrower Categories,Broader Categories,ConceptNet_Distances,ConceptNet_Relations
DBpedia URL,,,,,,,,,,,,
http://dbpedia.org/resource/Personal_knowledge_management,Personal knowledge management,Personal knowledge management (PKM) is a proce...,,[http://dbpedia.org/class/yago/WikicatInterdis...,[http://dbpedia.org/resource/Category:Knowledg...,[http://dbpedia.org/resource/Pattern_recogniti...,"[http://dbpedia.org/resource/Zettelkasten, htt...",[],[],[],"{'http://dbpedia.org/resource/Semantics': 0.0,...","{'/r/RelatedTo': [], '/r/IsA': [], '/r/HasCont..."
http://dbpedia.org/resource/Thought,Thought,"In their most common sense, the terms thought ...",http://commons.wikimedia.org/wiki/Special:File...,[http://www.w3.org/2002/07/owl#Thing],[http://dbpedia.org/resource/Category:Intellig...,[http://dbpedia.org/resource/Outline_of_though...,[http://dbpedia.org/resource/Thinking_Skills_a...,[],[],[],{'http://dbpedia.org/resource/Discipline': -0....,"{'/r/RelatedTo': ['/c/en/think', '/c/en/idea',..."
http://dbpedia.org/resource/Mind,Mind,The mind is the set of faculties responsible f...,http://commons.wikimedia.org/wiki/Special:File...,[http://www.w3.org/2002/07/owl#Thing],[http://dbpedia.org/resource/Category:A_priori...,[http://dbpedia.org/resource/Internalism_and_e...,"[http://dbpedia.org/resource/Katalemwa, http:/...",[],[],[],{'http://dbpedia.org/resource/Discipline': 0.0...,"{'/r/RelatedTo': ['/c/en/brain', '/c/en/thinki..."
http://dbpedia.org/resource/Brain,Brain,A brain is an organ that serves as the center ...,http://commons.wikimedia.org/wiki/Special:File...,"[http://www.w3.org/2002/07/owl#Thing, http://d...",[http://dbpedia.org/resource/Category:Human_an...,"[http://dbpedia.org/resource/Fatty_acid, http:...","[http://dbpedia.org/resource/Piotr_Popik, http...",[],[],[],{'http://dbpedia.org/resource/Discipline': -0....,"{'/r/RelatedTo': ['/c/en/head', '/c/en/attenti..."
http://dbpedia.org/resource/Internet,Internet,The Internet (or internet) is the global syste...,http://commons.wikimedia.org/wiki/Special:File...,"[http://www.w3.org/2002/07/owl#Thing, http://d...",[http://dbpedia.org/resource/Category:Internet...,"[http://dbpedia.org/resource/London, http://db...",[http://dbpedia.org/resource/KTU_radio_Gaudeam...,[],[],[],{'http://dbpedia.org/resource/Semantics': 0.05...,"{'/r/RelatedTo': ['/c/en/intranet', '/c/en/net..."


In [20]:
#concepts_df.head(2)
 
concept_links = {}

conceptnet_keys = list(conceptnet_to_dbpedia.keys())

for concept_url in concepts_list:
	concept_links[concept_url] = {'link_from': [], 'link_to': [], 'both_links': [], 'shared_subject':{}, 'conceptnet_relations': []}
	#print(concept_url)
	#print(concepts_df_dbpedia_index.loc[concept_url, "Name"])

for concept_url in concepts_list:

	concepts_linked_from_this_concept = concepts_df.loc[concept_url, "Related Entities FROM"]
	this_concept_is_linked_from = concepts_df.loc[concept_url, "Related Entities OF"]
	subject_list = concepts_df.loc[concept_url, "Subject FROM"]

	conceptnet_dict = concepts_df.loc[concept_url, "ConceptNet_Relations"] #ast.literal_eval(concepts_df.loc[concept_url, "ConceptNet_Relations"])

	for relation in conceptnet_dict.keys():
		related_concepts = conceptnet_dict[relation]
		for related_concept in related_concepts:
			if related_concept in conceptnet_keys and conceptnet_to_dbpedia[related_concept] != concept_url:
				concept_links[concept_url]['conceptnet_relations'].append([relation, conceptnet_to_dbpedia[related_concept]])

	for other_concept in concepts_list:

		

		if other_concept in concepts_linked_from_this_concept:
			concept_links[other_concept]['link_from'].append(concept_url)

		if other_concept in this_concept_is_linked_from:
			concept_links[other_concept]['link_to'].append(concept_url)

		if other_concept in concepts_linked_from_this_concept and other_concept in this_concept_is_linked_from:
			concept_links[other_concept]['both_links'].append(concept_url)

		other_subject_list = concepts_df.loc[other_concept, "Subject FROM"]
		if concept_url != other_concept:
			for subject in other_subject_list:

				if subject in subject_list:

					if concept_url in list(concept_links[other_concept]['shared_subject'].keys()):

						concept_links[other_concept]['shared_subject'][concept_url].append(subject)
					else: 
						concept_links[other_concept]['shared_subject'][concept_url]= [subject]
	


## Execute Concept Nodes

In [21]:
#def create_concept_md_file(name, url, abstract, image_url, concept_links):
for concept_url in concepts_list:
	
	url = concept_url
	name = concepts_df.loc[concept_url, "Name"]
	image_url = concepts_df.loc[concept_url, "Image"]
	abstract = concepts_df.loc[concept_url, "Abstract"]
	
	title = name.title()
	mdConcept = MdUtils(file_name=join(md_dir, "Concepts", title))

	mdConcept.new_paragraph("---")
	mdConcept.new_paragraph("tags: #Concept")
	mdConcept.new_paragraph("---")

	mdConcept.new_header(level=1, title=title)
	mdConcept.new_paragraph(abstract, wrap_width=0)
	
	if type(image_url) ==type(" "): # image_url != "":
		image = "![" + str(name) + "](" + image_url + ")"
		mdConcept.new_paragraph(image, wrap_width=0)

	mdConcept.new_paragraph("", wrap_width=0)
	mdConcept.new_header(level=2, title="Links")
	mdConcept.new_paragraph("Wikipedia URL : " + "https://en.wikipedia.org/wiki/" + "_".join(name.split(" ")))
	mdConcept.new_paragraph("DBpedia URL : " + url)
	mdConcept.new_paragraph("", wrap_width=0)

	mdConcept.new_header(level=2, title="Related Concepts")

	used_concepts = []

	relation_list = ["/r/Causes",
		"/r/CapableOf",
		"/r/MotivatedByGoal",
		"/r/HasPrerequisite",
		"/r/Synonym",
		"/r/SimilarTo",
			"/r/IsA",
			"/r/RelatedTo",
			"/r/Antonym",
			"/r/DistinctFrom",
				"/r/HasContext",
				"/r/HasProperty",
				"/r/PartOf",
				"/r/UsedFor",]

	conceptnet_relations = concept_links[url]['conceptnet_relations']

	for relation_type in relation_list:
		for pair in conceptnet_relations:
			relation = pair[0]
			concept = pair[1]
			if concept in concepts_list:
				concept_name = concepts_df.loc[concept, "Name"]
				if relation_type == relation:
					if concept not in used_concepts:
						mdConcept.new_paragraph("- " + relation.split("r/")[1] + " [[" + concept_name + "]]")
						used_concepts.append(concept)

	count=0	
	for concept_url in concept_links[url]['both_links']:
		concept_name = concepts_df.loc[concept_url, "Name"]
		if concept_url != url and count < 5 and concept_url not in used_concepts:
			count+=1
			mdConcept.new_paragraph("- HasWikiLink [[" + concept_name + "]]")
			used_concepts.append(concept_url)
	for concept_url in concept_links[url]['link_from']:
		concept_name = concepts_df.loc[concept_url, "Name"]
		if concept_url != url and count < 5 and concept_url not in used_concepts:
			count+=1
			mdConcept.new_paragraph("- HasWikiLink [[" + concept_name + "]]")
			used_concepts.append(concept_url)
	for concept_url in concept_links[url]['link_to']:
		concept_name = concepts_df.loc[concept_url, "Name"]
		if concept_url != url and count < 5 and concept_url not in used_concepts:
			count+=1
			mdConcept.new_paragraph("- HasWikiLink [[" + concept_name + "]]")
			used_concepts.append(concept_url)

	for concept2_url in used_concepts:
		#print(url, concept2_url)
		G.add_edge(url, concept2_url)
		G_only_all_concepts.add_edge(url, concept2_url)

		GD.add_edge(url, concept2_url)
		GD_only_all_concepts.add_edge(url, concept2_url)

		GD.add_edge(concept2_url, url)
		GD_only_all_concepts.add_edge(concept2_url, url)
		
	
	# for concept_subject_dict_key in concept_links[url]['shared_subject'].keys():

	# 	concept_name = concepts_df_dbpedia_index.loc[concept_subject_dict_key, "Name"]mdConcept.new_paragraph
	# 	mdConcept.new_paragraph("- shares_subject_with [[" + concept_name + "]]")
	# 	for subject in concept_links[url]['shared_subject'][concept_subject_dict_key]:
	# 		mdConcept.new_paragraph(" → " + subject.split("/Category:")[1].replace("_", " "))
	
	mdConcept.create_md_file()

In [22]:
"""for concept_url in concepts_list:
	
	name = concepts_df.loc[concept_url, "Name"]
	image_url = concepts_df.loc[concept_url, "Image"]
	abstract = concepts_df.loc[concept_url, "Abstract"]

	create_concept_md_file(name, concept_url, abstract, image_url, concept_links)
"""

'for concept_url in concepts_list:\n\t\n\tname = concepts_df.loc[concept_url, "Name"]\n\timage_url = concepts_df.loc[concept_url, "Image"]\n\tabstract = concepts_df.loc[concept_url, "Abstract"]\n\n\tcreate_concept_md_file(name, concept_url, abstract, image_url, concept_links)\n'

# Graph Analysis

In [23]:
text_nodes = list(text_df['Title'])

### Basic info

In [24]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.04480841168739258


In [25]:
nx.info(G)

/var/folders/m9/v96g17d503957kc60gl1ly4h0000gp/T/ipykernel_60114/1064119803.py:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  nx.info(G)


'Graph with 315 nodes and 2216 edges'

### Shortest Paths

In [26]:
graphsD = [GD, GD_only_texts, GD_only_text_concepts, GD_only_all_concepts]
graphs = [G, G_only_texts, G_only_text_concepts, G_only_all_concepts]

graph_dicts = []

for graph in graphs:

	dict_shortest_paths = {}

	text_shortest_paths = []


	for title in text_nodes:

		dict_shortest_paths[title] = {'avg':0, 'max': 0, 'all_paths':{} }
		
		all_shortest_paths = []

		local_max = 0
		count = 0
		for title2 in text_nodes:

			if nx.has_path(graph,title,title2) == True:
				distance = nx.shortest_path_length(graph,title,title2)
				path = nx.shortest_path(graph,title,title2)
				count += 1
			else:
				distance = -1
				path = []
			if distance > local_max:
				local_max = distance

			dict_shortest_paths[title]['all_paths'][title2] = [distance, path]

			all_shortest_paths.append(distance)
		
		dict_shortest_paths[title]['max'] = local_max
		dict_shortest_paths[title]['avg'] = sum(all_shortest_paths)/count

		text_shortest_paths.append(all_shortest_paths)
	
	graph_dicts.append(dict_shortest_paths)

In [27]:
data=[]

for title in text_nodes: 
	row = []
	for info_dict in graph_dicts:
		row.append(round(info_dict[title]['avg'], 3))
		row.append(info_dict[title]['max'])
	data.append(row)

[G, G_only_texts, G_only_text_concepts, G_only_all_concepts]

columns = ["Full Avg", "Full Max", "Only Text Avg", "Only Text Max", "Text-Concept Avg", "Text-Concept Max", "All Concepts Avg", "All Concepts Max"]

shortest_path_df = pd.DataFrame(data, index=text_nodes, columns=columns)	


In [28]:
len(shortest_path_df)

163

In [29]:
shortest_path_df[shortest_path_df["Full Max"]>4]

,Full Avg,Full Max,Only Text Avg,Only Text Max,Text-Concept Avg,Text-Concept Max,All Concepts Avg,All Concepts Max
Why Information Grows Note 26,2.834,5,2.847,5,-162.0,0,-162.0,0
The PARA Method Note 13,3.190,5,3.190,5,-162.0,0,-162.0,0


In [30]:

#text_nodes[position]

In [31]:
for i in range(len(text_nodes)):
	if [shortest_path_df["Full Max"]>4][0][i]:
		print(i, text_nodes[i])

83 Why Information Grows Note 26
111 The PARA Method Note 13


In [32]:
graph_dicts[1][text_nodes[165]]['all_paths']["The Almanack of Naval Ravikant Note 15"]

IndexError: list index out of range

In [ ]:
graph_dicts[0][text_nodes[179]]['all_paths']["Courage Is Calling Note 12"]

[5,
 ['How to Take Smart Notes Note 17',
  'Building a Second Brain Note 29',
  'Building a Second Brain Note 7',
  'Feel the Fear and Do It Anyway Note 29',
  'Feel the Fear and Do It Anyway Note 3',
  'Courage Is Calling Note 12']]

In [ ]:
position = 179 #83 #179

max_distance = 5

text1_name = text_nodes[position]

print(text1_name, "\n")



print(raw_text_df.iloc[position,3], "\n\n----------------------------- \n")

for item in graph_dicts[0][text1_name]['all_paths'].items():
	if item[1][0] == max_distance:
		text2_name = item[0]
		print(text2_name, "\n")
		text2 = raw_text_df[text_df["Title"]==item[0]].iloc[0,3]
		print(text2, "\n")
		text2_id = raw_text_df[text_df["Title"]==item[0]].iloc[0,-1]
		print("Relatedness:", str(relatedness_semantic_df.iloc[position][str(text2_id)]))


How to Take Smart Notes Note 17 

The first type of links are those on notes that are giving you the overview of a topic. These are notes directly referred to from the index and usually used as an entry point into a topic that has already developed to such a degree that an overview is needed or at least becomes helpful. On a note like this, you can collect links to other relevant notes to this topic or question, preferably with a short indication of what to find on these notes 

----------------------------- 

The Almanack of Naval Ravikant Note 6 

Pick an industry where you can play long-term games with long-term people. 

Relatedness: -0.0805
The Almanack of Naval Ravikant Note 9 

Learn to sell. Learn to build. If you can do both, you will be unstoppable. 

Relatedness: -0.06178
The Almanack of Naval Ravikant Note 21 

Work as hard as you can. Even though who you work with and what you work on are more important than how hard you work. 

Relatedness: 0.00817
The Almanack of Naval R

### Egde Degrees

In [33]:
graphs = [G, G_only_texts, G_only_text_concepts, G_only_all_concepts]

degrees_list = []

text_data = []

for i in range(4):
	degrees_list.append(graphs[i].degree)

for title in text_nodes:
	row = []
	for i in range(4):
		degree = degrees_list[i][title]
		row.append(degree)
	text_data.append(row)

concept_data = []

for concept in concepts_list:
	row = []
	for i in [0,2,3]:
		degree = degrees_list[i][concept]
		row.append(degree)
	concept_data.append(row)

concept_degree_df = pd.DataFrame(concept_data, index=concepts_list, columns=["Full Degree", "Text-Concept Degree", "All Concepts Degree"])

text_degree_df = pd.DataFrame(text_data, index=text_nodes, columns=["Full Degree", "Only Text Degree", "Text-Concept Degree", "All Concepts Degree"])

In [34]:
concept_degree_df.head()

,Full Degree,Text-Concept Degree,All Concepts Degree
http://dbpedia.org/resource/Personal_knowledge_management,9,2,9
http://dbpedia.org/resource/Thought,43,19,43
http://dbpedia.org/resource/Mind,42,9,42
http://dbpedia.org/resource/Brain,30,8,30
http://dbpedia.org/resource/Internet,15,3,15


In [35]:
concept_degree_df.mean()

Full Degree            15.244094
Text-Concept Degree     4.692913
All Concepts Degree    15.244094
dtype: float64

In [36]:
concept_degree_df.min()

Full Degree            2
Text-Concept Degree    2
All Concepts Degree    2
dtype: int64

In [37]:
text_degree_df.head()

,Full Degree,Only Text Degree,Text-Concept Degree,All Concepts Degree
How to Take Smart Notes Note 1,21,16,5,5
How to Take Smart Notes Note 2,7,7,0,0
How to Take Smart Notes Note 3,13,11,2,2
How to Take Smart Notes Note 4,21,16,5,5
How to Take Smart Notes Note 5,11,11,0,0


In [38]:
text_degree_df.mean()

Full Degree            14.938650
Only Text Degree       10.907975
Text-Concept Degree     4.030675
All Concepts Degree     4.030675
dtype: float64

In [39]:
text_degree_df.min()

Full Degree            4
Only Text Degree       4
Text-Concept Degree    0
All Concepts Degree    0
dtype: int64

In [40]:
isolated = list(text_degree_df.sort_values(by=["Full Degree"]).index)[0:10]
pd.concat([shortest_path_df, text_degree_df, text_df.set_index('Title')['Text']], axis=1).sort_values(by="Full Degree").loc[isolated].iloc[0,-1]

'PROMISE 5: You will beat information overload and FOMO: The fear of missing out on a key piece of information will disappear and be replaced with the confidence that you have everything you need to get started.'

In [366]:
print(sorted(G.degree, key=lambda x: x[1], reverse=True)[0:20])

[('http://dbpedia.org/resource/Fear', 40), ('http://dbpedia.org/resource/Life', 39), ('http://dbpedia.org/resource/Emotion', 36), ('http://dbpedia.org/resource/Mind', 32), ('http://dbpedia.org/resource/Person', 32), ('http://dbpedia.org/resource/Education', 27), ('http://dbpedia.org/resource/Thought', 26), ('http://dbpedia.org/resource/Experience', 26), ('http://dbpedia.org/resource/Learning', 25), ('http://dbpedia.org/resource/Idea', 23), ('http://dbpedia.org/resource/Knowledge', 23), ('Feel the Fear and Do It Anyway Note 1', 22), ('Courage Is Calling Note 16', 22), ('http://dbpedia.org/resource/Depression_(mood)', 22), ('http://dbpedia.org/resource/Love', 22), ('http://dbpedia.org/resource/Truth', 22), ('Feel the Fear and Do It Anyway Note 17', 21), ('http://dbpedia.org/resource/Feeling', 21), ('http://dbpedia.org/resource/Virtue', 21), ('Courage Is Calling Note 2', 20)]


### Node Importance

In [ ]:
pageranks = nx.pagerank(G) # A dictionary
#print(pageranks)
#print(sorted(pageranks, key=lambda x: x[1], reverse=True)[0:20])



### Node Importance

In [ ]:
pageranks = nx.pagerank(G) # A dictionary
#print(pageranks)
#print(sorted(pageranks, key=lambda x: x[1], reverse=True)[0:20])



In [92]:
betweenness_dict = nx.betweenness_centrality(G) # Run betweenness centrality
eigenvector_dict = nx.eigenvector_centrality(G) # Run eigenvector centrality

nx.set_node_attributes(G, betweenness_dict, 'betweenness')
nx.set_node_attributes(G, eigenvector_dict, 'eigenvector')

print(sorted(betweenness_dict, key=lambda x: x[1], reverse=True)[0:20])

print(sorted(eigenvector_dict, key=lambda x: x[1], reverse=True)[0:20])

['http://dbpedia.org/resource/Fear', 'http://dbpedia.org/resource/Mental_disorder', 'http://dbpedia.org/resource/Education', 'http://dbpedia.org/resource/Mind', 'http://dbpedia.org/resource/Fact', 'http://dbpedia.org/resource/Risk', 'http://dbpedia.org/resource/Nothing', 'http://dbpedia.org/resource/Other_(philosophy)', 'http://dbpedia.org/resource/Paralysis', 'http://dbpedia.org/resource/Power_(social_and_political)', 'http://dbpedia.org/resource/Choice', 'http://dbpedia.org/resource/Energy', 'http://dbpedia.org/resource/Action_(philosophy)', 'http://dbpedia.org/resource/Pain', 'http://dbpedia.org/resource/Depression_(mood)', 'http://dbpedia.org/resource/Moral_responsibility', 'http://dbpedia.org/resource/Awareness', 'http://dbpedia.org/resource/Sadness', 'http://dbpedia.org/resource/Evidence', 'http://dbpedia.org/resource/Blame']
['http://dbpedia.org/resource/Fear', 'http://dbpedia.org/resource/Mental_disorder', 'http://dbpedia.org/resource/Education', 'http://dbpedia.org/resource/Mi

In [90]:
from networkx.algorithms.community.label_propagation import label_propagation_communities

communities = label_propagation_communities(G)
print([community for community in communities])

[{'http://dbpedia.org/resource/Choice', 'http://dbpedia.org/resource/Evidence', 'Feel the Fear and Do It Anyway Note 16', 'Courage Is Calling Note 11', 'Feel the Fear and Do It Anyway Note 28', 'Feel the Fear and Do It Anyway Note 24', 'Feel the Fear and Do It Anyway Note 30', 'http://dbpedia.org/resource/Knowledge', 'Feel the Fear and Do It Anyway Note 4', 'Feel the Fear and Do It Anyway Note 18', 'Courage Is Calling Note 14', 'Courage Is Calling Note 3', 'Courage Is Calling Note 16', 'http://dbpedia.org/resource/Courage', 'Courage Is Calling Note 2', 'Feel the Fear and Do It Anyway Note 22', 'http://dbpedia.org/resource/Action_(philosophy)', 'Feel the Fear and Do It Anyway Note 13', 'Feel the Fear and Do It Anyway Note 11', 'Feel the Fear and Do It Anyway Note 23', 'Courage Is Calling Note 8', 'http://dbpedia.org/resource/Excellence', 'Courage Is Calling Note 17', 'Courage Is Calling Note 4', 'http://dbpedia.org/resource/Person', 'http://dbpedia.org/resource/Pain', 'http://dbpedia.or